In [36]:
import numpy as np

def laplace_gamma_dp(weights, shape_param, x_values, y_values):
    """
    Thêm nhiễu vi sai bằng phân phối Laplace Gamma vào trọng số mô hình.
    """
    delta = np.max(np.abs(x_values - y_values))
    lambda_param = delta / np.mean(x_values) if np.mean(x_values) != 0 else 1.0
    
    print(delta)
    
    print("----")
    print(lambda_param)
    
    gamma_1 = np.random.gamma(shape_param, lambda_param, size=weights.shape)
    gamma_2 = np.random.gamma(shape_param, lambda_param, size=weights.shape)
    laplace_gamma_noise = gamma_1 - gamma_2
    encrypted_weights = weights + laplace_gamma_noise
    return encrypted_weights

def diffie_hellman_offset(encrypted_weights, client_id, active_clients, deltas):
    """
    Áp dụng lớp mã hóa thứ hai bằng phương pháp trao đổi khóa Diffie-Hellman.
    """
    offset = 0
    add = True
    
    for cid in active_clients:
        if cid == client_id:
            add = False
        
        if add:
            offset += deltas.get(cid, 0)
        else:
            offset -= deltas.get(cid, 0)
    
    offset_encrypted_weights = encrypted_weights + offset
    return offset_encrypted_weights

def homomorphic_encryption(offset_encrypted_weights):
    """
    Áp dụng lớp mã hóa thứ ba bằng phương pháp Homomorphic Encryption (HE).
    """
    operations = ['add', 'subtract', 'multiply', 'divide']
    operation = np.random.choice(operations)
    factor = np.random.randint(1, 10)
    
    if operation == 'divide' and np.any(offset_encrypted_weights == 0):
        operation = np.random.choice(['add', 'subtract', 'multiply'])
    
    he_encrypted_weights = np.copy(offset_encrypted_weights)
    
    for i in range(he_encrypted_weights.shape[0]):
        for j in range(he_encrypted_weights.shape[1]):
            value = he_encrypted_weights[i, j]
            if operation == 'add':
                he_encrypted_weights[i, j] = value + factor
            elif operation == 'subtract':
                he_encrypted_weights[i, j] = value - factor
            elif operation == 'multiply':
                he_encrypted_weights[i, j] = value * factor
            elif operation == 'divide':
                he_encrypted_weights[i, j] = value / factor if factor != 0 else value
    
    return he_encrypted_weights

# Ví dụ sử dụng:
np.random.seed(42)
weights = np.array([[0.5, -0.2], [0.3, 0.8]])
x_values = np.array([1.0, 2.0, 3.0, 4.0])
y_values = np.array([1.1, 1.9, 3.2, 3.8])
shape_param = 2.0

# Lớp mã hóa đầu tiên (DP Laplace Gamma)
encrypted_weights = laplace_gamma_dp(weights, shape_param, x_values, y_values)

# Giả lập danh sách client và giá trị delta
active_clients = ["A", "B", "C", "D"]
deltas = {"A": 1.5, "B": -0.5, "C": 2.0, "D": -1.0}
client_id = "C"

# Lớp mã hóa thứ hai (DK Offset)
offset_encrypted_weights = diffie_hellman_offset(encrypted_weights, client_id, active_clients, deltas)

# Lớp mã hóa thứ ba (HE Encryption)
final_encrypted_weights = homomorphic_encryption(offset_encrypted_weights)

print("Original Weights:")
print(weights)
print("\nEncrypted Weights with DP Noise:")
print(encrypted_weights)
print("\nFinal Encrypted Weights with DK Offset:")
print(offset_encrypted_weights)
print("\nFinal Encrypted Weights with HE:")
print(final_encrypted_weights)


0.20000000000000018
----
0.08000000000000007
Original Weights:
[[ 0.5 -0.2]
 [ 0.3  0.8]]

Encrypted Weights with DP Noise:
[[ 0.3195172  -0.30977932]
 [ 0.32009645  0.71299903]]

Final Encrypted Weights with DK Offset:
[[ 0.3195172  -0.30977932]
 [ 0.32009645  0.71299903]]

Final Encrypted Weights with HE:
[[ 0.05325287 -0.05162989]
 [ 0.05334941  0.11883317]]
